In [10]:
%AddDeps org.apache.spark spark-avro_2.12 3.3.2 --transitive

Marking org.apache.spark:spark-avro_2.12:3.3.2 for download
Obtained 12 files


lastException = null


Marking org.apache.spark:spark-avro_2.12:3.3.2 for download
Obtained 12 files


In [11]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("Transfer File from GCS to HDFS")
  .getOrCreate()

val gcsPath = "gs://first-job-bucket/movies.csv"
val hdfsPath = "hdfs:///spark/Day16_17/CS4/movies.csv"

val data = spark.read
  .option("header", "true")
  .csv(gcsPath)

data.write
  .option("header", "true")
  .mode("overwrite")
  .csv(hdfsPath)

println(s"File transferred from $gcsPath to $hdfsPath successfully!")

File transferred from gs://first-job-bucket/movies.csv to hdfs:///spark/Day16_17/CS4/movies.csv successfully!


spark = org.apache.spark.sql.SparkSession@108ea4e2
gcsPath = gs://first-job-bucket/movies.csv
hdfsPath = hdfs:///spark/Day16_17/CS4/movies.csv
data = [movieId: string, title: string ... 1 more field]


[movieId: string, title: string ... 1 more field]

In [12]:
val moviesPath = "hdfs:///spark/Day16_17/CS4/movies.csv"
val moviesDF = spark.read.option("header", "true").csv(moviesPath)

val sampleMoviesDF = moviesDF.limit(1000)
val duplicateMoviesDF = moviesDF.union(sampleMoviesDF)

duplicateMoviesDF.write
  .option("header", "true")
  .mode("overwrite")
  .csv("hdfs:///spark/Day16_17/CS4/duplicated_movies.csv")

println("1000 duplicates inserted and file updated successfully!")

1000 duplicates inserted and file updated successfully!


moviesPath = hdfs:///spark/Day16_17/CS4/movies.csv
moviesDF = [movieId: string, title: string ... 1 more field]
sampleMoviesDF = [movieId: string, title: string ... 1 more field]
duplicateMoviesDF = [movieId: string, title: string ... 1 more field]


[movieId: string, title: string ... 1 more field]

In [13]:
val moviesPath = "hdfs:///spark/Day16_17/CS4/duplicated_movies.csv"
val moviesDF = spark.read.option("header", "true").csv(moviesPath)

val cleanedMoviesDF = moviesDF.rdd.map(row => {
  val movieId = row.getString(row.fieldIndex("movieId"))
  val title = row.getString(row.fieldIndex("title"))
  val genres = row.getString(row.fieldIndex("genres"))
  ((movieId, title), genres)
}).reduceByKey((genres1, genres2) => s"$genres1|$genres2").map {
  case ((movieId, title), combinedGenres) => (movieId, title, combinedGenres)
}.toDF("movieId", "title", "genres")

val originalCount = moviesDF.count()
val deduplicatedCount = cleanedMoviesDF.count()
val duplicatesRemoved = originalCount - deduplicatedCount
println(s"Original record count: $originalCount")
println(s"Deduplicated record count: $deduplicatedCount")
println(s"Duplicates removed: $duplicatesRemoved")

Original record count: 88585
Deduplicated record count: 87585
Duplicates removed: 1000


moviesPath = hdfs:///spark/Day16_17/CS4/duplicated_movies.csv
moviesDF = [movieId: string, title: string ... 1 more field]
cleanedMoviesDF = [movieId: string, title: string ... 1 more field]
originalCount = 88585
deduplicatedCount = 87585
duplicatesRemoved = 1000


1000

In [17]:
val outputPath = s"gs://first-job-bucket/cleaned_movies.parquet"
cleanedMoviesDF.write
  .format("parquet")
  .mode("overwrite")
  .save(outputPath)

println("Cleaned movies data saved successfully in parquet format.")

Cleaned movies data saved successfully in parquet format.


lastException = null
outputPath = gs://first-job-bucket/cleaned_movies.parquet


gs://first-job-bucket/cleaned_movies.parquet